imports

In [2]:
import pandas as pd
import torch
import numpy as np
# import torchvision
import matplotlib.pyplot as plt
import time
import copy
from torch.utils.data import Dataset
import shutil 
from tqdm import tqdm

In [3]:
# pd.read_csv("data_for_competition/test_target.csv")

In [3]:
file_names = [f'data_for_competition/train_data/train_data_{i}.pq' for i in range(12)]
# file_names

In [4]:
# s=0
# for i in range(12):
#     df = pd.read_parquet(f'data_for_competition/train_data/train_data_{i}.pq', engine='fastparquet')#[:100]
#     s+=df.shape[0]
#     print(df.groupby("id").rn.count().max())
# # print(s)

In [22]:
df = pd.read_parquet(f'data_for_competition/train_data/train_data_{0}.pq', engine='fastparquet')[:100]
df.id

0      0
1      0
2      0
3      0
4      0
      ..
95    11
96    11
97    12
98    12
99    12
Name: id, Length: 100, dtype: int64

In [15]:
a = np.array([[1.,  1.,  1.,  1.,  1.],
              [1.,  1.,  1.,  1.,  1.],
              [1.,  1.,  1.,  1.,  1.]])
np.pad(a, [(0, 10-a.shape[0]), (0, 0)], mode='constant')


array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [68]:
def get_padded_tensor(df, orders_cnt=70):
    tensors = []
    ids =[]
    for id, order in df.groupby("id"):
        order = order.sort_values("rn").drop('rn', axis=1).values
        tensors.append(torch.tensor(np.pad(order, [(0, orders_cnt-order.shape[0]), (0, 0)], mode='constant')))
        # ids.append(id)
    return torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)


In [17]:
# df[:30]

Loader

In [8]:
class CustomDataset(Dataset):
    def __init__(self, filenames, sample):
      # `filenames` is a list of strings that contains all file names.
      # `batch_size` determines the number of files that we want to read in a chunk.
        self.filenames = filenames
        self.sample = sample

    def __len__(self):
        # Number of chunks.
        return len(self.filenames)

    def __getitem__(self, idx):  # idx means index of the chunk.
      # In this method, we do all the preprocessing.
      # First read  data from files in a chunk. Preprocess it. Extract labels. Then return data and labels.
        # This extracts one batch of file names from the list `filenames`.
        file = self.filenames[idx]
        data = pd.read_parquet(file, engine='fastparquet')
        cat_columns = list(data.columns)[2:]
        data[cat_columns] = data[cat_columns].astype('category')

        # ids, data = get_padded_tensor(data)

        target_file_name = f'data_for_competition/{self.sample}_target.csv'
        target = pd.read_csv(target_file_name)
        labels = target[target.id.isin(set(data.id))]
        # The following condition is actually needed in Pytorch. Otherwise, for our particular example, the iterator will be an infinite loop.
        # Readers can verify this by removing this condition.
        if idx == self.__len__():
            raise IndexError

        return data, labels


In [9]:
train_dataloader = CustomDataset([f'data_for_competition/train_data/train_data_{i}.pq' for i in range(12)],
                                  'train')
test_dataloader = CustomDataset([f'data_for_competition/test_data/test_data_{i}.pq' for i in range(2)],
                                  'test')
val_dataloader = test_dataloader

In [10]:
# for data, labels in ds:
#     print(data.shape)
#     # print(labels.shape)
#     break

In [20]:
inputs, labels  = train_dataloader[0]

In [40]:
data.iloc[3]

id                         0
rn                         4
pre_since_opened           4
pre_since_confirmed        1
pre_pterm                  9
                          ..
enc_loans_credit_status    3
enc_loans_credit_type      1
enc_loans_account_cur      1
pclose_flag                0
fclose_flag                0
Name: 3, Length: 61, dtype: int64

In [58]:
def get_batches(inputs, labels, batch_size=10_000):
    """batch_size - кол-во продуктов-записей"""
    if len(inputs) <= batch_size:
        return [inputs], [labels]
    res_batches = []
    res_labels = []
    l = 0
    last_id =-1
    while last_id != labels.id.iloc[-1]:
        next_id_ind = inputs[inputs.id > last_id].index[0]
        l = inputs.index.get_loc(next_id_ind)
        first_id = inputs.iloc[l].id
        if l+batch_size>=len(inputs):
            last_id = inputs.id.iloc[-1]
        else:
            last_id = inputs.iloc[l+batch_size].id
        batch = inputs[(inputs.id >= first_id) & (inputs.id <= last_id)]
        res_batches.append(batch)
        res_labels.append(labels[labels.id.isin(set(batch.id))])        
    return res_batches, res_labels


In [50]:
# inputs[:30]

In [61]:
b_inp, b_lab = get_batches(inputs, labels , batch_size=10_000)

In [63]:
batch=b_inp[0]
label = b_lab[0]

In [69]:
tensor = get_padded_tensor(batch)

In [78]:
tensor.shape[2]

60

In [73]:
import torch.nn as nn
from torch.autograd import Variable 

class LstmClassifier(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LstmClassifier, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

(1270, 70, 60)

In [79]:
model = LstmClassifier(num_classes=2,
                       input_size=tensor.shape[2],
                       hidden_size=2,
                       num_layers=1,
                       seq_length=None)


In [80]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [82]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for huge_inputs, huge_labels in tqdm(dataloader[:1]):
                batch_inputs, batch_labels = get_batches(
                    huge_inputs, huge_labels, batch_size=10_000)
                batch_cnt = 1
                for inputs, labels in zip(batch_inputs[:batch_cnt], batch_labels[:batch_cnt]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    # forward and backward
                    with torch.set_grad_enabled(phase == 'train'):
                        preds = model(inputs)
                        loss_value = loss(preds, labels)
                        preds_class = preds.argmax(dim=1)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss_value.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss_value.item()
                    running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc), flush=True)

    return model


In [1]:
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)
train_model(model,
            torch.nn.CrossEntropyLoss(),
            optimizer,
            torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1),
            1)


NameError: name 'torch' is not defined